# Import

In [1]:
import pandas
import sklearn
import evaluation
import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
GPU_fix = "MemoryFix" # Choose "Disable", "MemoryFix" or "None"

if GPU_fix == "Disable":
    try:
        # Disable all GPUS
        tf.config.set_visible_devices([], 'GPU')
        visible_devices = tf.config.get_visible_devices()
        for device in visible_devices:
            assert device.device_type != 'GPU'
    except:
        # Invalid device or cannot modify virtual devices once initialized.
        pass
elif GPU_fix == "MemoryFix":
    gpus = tf.config.experimental.list_physical_devices('GPU')
    if gpus:
        try:
            # Currently, memory growth needs to be the same across GPUs
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            # Memory growth must be set before GPUs have been initialized
            print(e)

1 Physical GPUs, 1 Logical GPUs


# Read training data

In [3]:
# https://www.kaggle.com/sionek/ugbc-gs
folder = 'tau_data/'
data = pandas.read_csv(folder + 'training.csv.zip', index_col='id')
#--------------- feature engineering -------------- #
def add_features(df):
    # features used by the others on Kaggle
    df['NEW_FD_SUMP']=df['FlightDistance']/(df['p0_p']+df['p1_p']+df['p2_p'])
    df['NEW5_lt']=df['LifeTime']*(df['p0_IP']+df['p1_IP']+df['p2_IP'])/3
    df['p_track_Chi2Dof_MAX'] = df.loc[:, ['p0_track_Chi2Dof', 'p1_track_Chi2Dof', 'p2_track_Chi2Dof']].max(axis=1)
    #df['flight_dist_sig'] = df['FlightDistance']/df['FlightDistanceError'] # modified to:
    df['flight_dist_sig2'] = (df['FlightDistance']/df['FlightDistanceError'])**2
    # features from phunter
    df['flight_dist_sig'] = df['FlightDistance']/df['FlightDistanceError']
    df['NEW_IP_dira'] = df['IP']*df['dira']
    df['p0p2_ip_ratio']=df['IP']/df['IP_p0p2']
    df['p1p2_ip_ratio']=df['IP']/df['IP_p1p2']
    df['DCA_MAX'] = df.loc[:, ['DOCAone', 'DOCAtwo', 'DOCAthree']].max(axis=1)
    df['iso_bdt_min'] = df.loc[:, ['p0_IsoBDT', 'p1_IsoBDT', 'p2_IsoBDT']].min(axis=1)
    df['iso_min'] = df.loc[:, ['isolationa', 'isolationb', 'isolationc','isolationd', 'isolatione', 'isolationf']].min(axis=1)
    # My:
    # new combined features just to minimize their number;
    # their physical sense doesn't matter
    df['NEW_iso_abc'] = df['isolationa']*df['isolationb']*df['isolationc']
    df['NEW_iso_def'] = df['isolationd']*df['isolatione']*df['isolationf']
    df['NEW_pN_IP'] = df['p0_IP']+df['p1_IP']+df['p2_IP']
    df['NEW_pN_p']  = df['p0_p']+df['p1_p']+df['p2_p']
    df['NEW_IP_pNpN'] = df['IP_p0p2']*df['IP_p1p2']
    df['NEW_pN_IPSig'] = df['p0_IPSig']+df['p1_IPSig']+df['p2_IPSig']
    #My:
    # "super" feature changing the result from 0.988641 to 0.991099
    df['NEW_FD_LT']=df['FlightDistance']/df['LifeTime']
    return df

print("Add features")
train_df = add_features(data)

print("Eliminate features")
filter_out = ['id', 'min_ANNmuon', 'production', 'mass', 'signal',
              'SPDhits','CDF1', 'CDF2', 'CDF3',
              'p0_pt', 'p1_pt', 'p2_pt',
              'p0_p', 'p1_p', 'p2_p', 'p0_eta', 'p1_eta', 'p2_eta',
              'isolationa', 'isolationb', 'isolationc', 'isolationd', 'isolatione', 'isolationf',
              'p0_IsoBDT', 'p1_IsoBDT', 'p2_IsoBDT',
              'p0_IP', 'p1_IP', 'p2_IP',
              'IP_p0p2', 'IP_p1p2',
              'p0_track_Chi2Dof', 'p1_track_Chi2Dof', 'p2_track_Chi2Dof',
              'p0_IPSig', 'p1_IPSig', 'p2_IPSig',
              'DOCAone', 'DOCAtwo', 'DOCAthree']

Add features
Eliminate features


In [4]:
#modified
variables_mod = list(f for f in data.columns if f not in filter_out)
print(len(variables_mod))

28


In [5]:
train_df = data[variables_mod]

In [6]:
msk = np.random.rand(len(train_df)) < 0.8
train = train_df[msk]
val = train_df[~msk]

In [7]:
print('validation samples:',len(val))
print('train samples:',len(train))

validation samples: 13528
train samples: 54025


# Baseline training

In [8]:
#Create training arrays
x_train = train
y_train = data['signal'][msk].to_numpy()
y_train = np.expand_dims(y_train,1)
x_val = val
y_val = data['signal'][~msk].to_numpy()
y_val = np.expand_dims(y_val,1)

Подключение LR Scheduler

In [9]:
def scheduler(epoch, lr):
    if epoch % 300 != 0:
        return lr
    else:
        return lr/5

Создание модели

In [14]:
model1m = tf.keras.models.Sequential([
  tf.keras.layers.InputLayer(len(variables_mod)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(200, activation=tf.keras.layers.LeakyReLU(alpha=0.2)), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(100, activation=tf.keras.layers.LeakyReLU(alpha=0.2)), 
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(50, activation=tf.keras.layers.LeakyReLU(alpha=0.2)),  
  tf.keras.layers.Dense(1, activation = tf.keras.activations.sigmoid)
])
model1m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [11]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

In [13]:

history = model1m.fit(x_train, y_train, epochs = 200, batch_size = 128, validation_data = (x_val,y_val))

Epoch 1/200
423/423 [==============================] - 8s 19ms/step - loss: 0.4131 - accuracy: 0.8163 - val_loss: 0.3551 - val_accuracy: 0.8467
Epoch 2/200
423/423 [==============================] - 7s 17ms/step - loss: 0.3862 - accuracy: 0.8309 - val_loss: 0.3441 - val_accuracy: 0.8480
Epoch 3/200
423/423 [==============================] - 8s 20ms/step - loss: 0.3844 - accuracy: 0.8314 - val_loss: 0.3504 - val_accuracy: 0.8491
Epoch 4/200
423/423 [==============================] - 9s 21ms/step - loss: 0.3782 - accuracy: 0.8338 - val_loss: 0.3447 - val_accuracy: 0.8505
Epoch 5/200
423/423 [==============================] - 9s 21ms/step - loss: 0.3760 - accuracy: 0.8349 - val_loss: 0.3410 - val_accuracy: 0.8522
Epoch 6/200
423/423 [==============================] - 8s 20ms/step - loss: 0.3749 - accuracy: 0.8361 - val_loss: 0.3427 - val_accuracy: 0.8516
Epoch 7/200
423/423 [==============================] - 8s 19ms/step - loss: 0.3691 - accuracy: 0.8394 - val_loss: 0.3402 - val_accuracy:

Epoch 58/200
423/423 [==============================] - 2s 5ms/step - loss: 0.3600 - accuracy: 0.8431 - val_loss: 0.3366 - val_accuracy: 0.8584
Epoch 59/200
423/423 [==============================] - 2s 5ms/step - loss: 0.3583 - accuracy: 0.8458 - val_loss: 0.3329 - val_accuracy: 0.8579
Epoch 60/200
423/423 [==============================] - 2s 5ms/step - loss: 0.3631 - accuracy: 0.8413 - val_loss: 0.3377 - val_accuracy: 0.8573
Epoch 61/200
423/423 [==============================] - 2s 5ms/step - loss: 0.3596 - accuracy: 0.8436 - val_loss: 0.3359 - val_accuracy: 0.8564
Epoch 62/200
423/423 [==============================] - 2s 5ms/step - loss: 0.3590 - accuracy: 0.8459 - val_loss: 0.3332 - val_accuracy: 0.8585
Epoch 63/200
423/423 [==============================] - 2s 5ms/step - loss: 0.3582 - accuracy: 0.8441 - val_loss: 0.3339 - val_accuracy: 0.8561
Epoch 64/200
423/423 [==============================] - 2s 5ms/step - loss: 0.3589 - accuracy: 0.8438 - val_loss: 0.3374 - val_accuracy:

Epoch 115/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3571 - accuracy: 0.8467 - val_loss: 0.3307 - val_accuracy: 0.8578
Epoch 116/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3562 - accuracy: 0.8458 - val_loss: 0.3339 - val_accuracy: 0.8567
Epoch 117/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3573 - accuracy: 0.8462 - val_loss: 0.3349 - val_accuracy: 0.8571
Epoch 118/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3571 - accuracy: 0.8451 - val_loss: 0.3333 - val_accuracy: 0.8579
Epoch 119/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3565 - accuracy: 0.8458 - val_loss: 0.3340 - val_accuracy: 0.8562
Epoch 120/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3564 - accuracy: 0.8456 - val_loss: 0.3368 - val_accuracy: 0.8576
Epoch 121/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3561 - accuracy: 0.8452 - val_loss: 0.3349 - val_ac

423/423 [==============================] - 2s 4ms/step - loss: 0.3555 - accuracy: 0.8451 - val_loss: 0.3338 - val_accuracy: 0.8583
Epoch 172/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3558 - accuracy: 0.8447 - val_loss: 0.3359 - val_accuracy: 0.8575
Epoch 173/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3550 - accuracy: 0.8453 - val_loss: 0.3326 - val_accuracy: 0.8581
Epoch 174/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3533 - accuracy: 0.8458 - val_loss: 0.3344 - val_accuracy: 0.8579
Epoch 175/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3555 - accuracy: 0.8457 - val_loss: 0.3333 - val_accuracy: 0.8570
Epoch 176/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3546 - accuracy: 0.8461 - val_loss: 0.3324 - val_accuracy: 0.8562
Epoch 177/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3523 - accuracy: 0.8490 - val_loss: 0.3306 - val_accuracy: 0.8583

In [15]:
callback = tf.keras.callbacks.LearningRateScheduler(scheduler)
model1m.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])
history2 = model1m.fit(x_train, y_train, epochs = 200, batch_size = 128, validation_data = (x_val,y_val), callbacks = [callback])

Epoch 1/200
423/423 [==============================] - 2s 5ms/step - loss: 0.4564 - accuracy: 0.7933 - val_loss: 0.3615 - val_accuracy: 0.8442
Epoch 2/200
423/423 [==============================] - 2s 4ms/step - loss: 0.4093 - accuracy: 0.8179 - val_loss: 0.3501 - val_accuracy: 0.8476
Epoch 3/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3979 - accuracy: 0.8252 - val_loss: 0.3521 - val_accuracy: 0.8467
Epoch 4/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3924 - accuracy: 0.8276 - val_loss: 0.3503 - val_accuracy: 0.8483
Epoch 5/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3877 - accuracy: 0.8292 - val_loss: 0.3479 - val_accuracy: 0.8468
Epoch 6/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3847 - accuracy: 0.8322 - val_loss: 0.3460 - val_accuracy: 0.8479
Epoch 7/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3848 - accuracy: 0.8330 - val_loss: 0.3451 - val_accuracy: 0.8488

Epoch 58/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3608 - accuracy: 0.8446 - val_loss: 0.3329 - val_accuracy: 0.8567
Epoch 59/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3602 - accuracy: 0.8438 - val_loss: 0.3309 - val_accuracy: 0.8574
Epoch 60/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3622 - accuracy: 0.8424 - val_loss: 0.3348 - val_accuracy: 0.8556
Epoch 61/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3615 - accuracy: 0.8427 - val_loss: 0.3343 - val_accuracy: 0.8556
Epoch 62/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3631 - accuracy: 0.8423 - val_loss: 0.3358 - val_accuracy: 0.8562
Epoch 63/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3605 - accuracy: 0.8449 - val_loss: 0.3330 - val_accuracy: 0.8564
Epoch 64/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3607 - accuracy: 0.8433 - val_loss: 0.3342 - val_accuracy:

Epoch 115/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3582 - accuracy: 0.8462 - val_loss: 0.3316 - val_accuracy: 0.8575
Epoch 116/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3586 - accuracy: 0.8447 - val_loss: 0.3323 - val_accuracy: 0.8576
Epoch 117/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3571 - accuracy: 0.8459 - val_loss: 0.3321 - val_accuracy: 0.8571
Epoch 118/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3596 - accuracy: 0.8423 - val_loss: 0.3314 - val_accuracy: 0.8578
Epoch 119/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3578 - accuracy: 0.8443 - val_loss: 0.3327 - val_accuracy: 0.8579
Epoch 120/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3584 - accuracy: 0.8442 - val_loss: 0.3309 - val_accuracy: 0.8584
Epoch 121/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3555 - accuracy: 0.8460 - val_loss: 0.3335 - val_ac

423/423 [==============================] - 2s 4ms/step - loss: 0.3567 - accuracy: 0.8454 - val_loss: 0.3326 - val_accuracy: 0.8577
Epoch 172/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3567 - accuracy: 0.8449 - val_loss: 0.3322 - val_accuracy: 0.8570
Epoch 173/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3580 - accuracy: 0.8446 - val_loss: 0.3315 - val_accuracy: 0.8578
Epoch 174/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3551 - accuracy: 0.8468 - val_loss: 0.3335 - val_accuracy: 0.8574
Epoch 175/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3555 - accuracy: 0.8463 - val_loss: 0.3313 - val_accuracy: 0.8573
Epoch 176/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3566 - accuracy: 0.8449 - val_loss: 0.3310 - val_accuracy: 0.8582
Epoch 177/200
423/423 [==============================] - 2s 4ms/step - loss: 0.3562 - accuracy: 0.8460 - val_loss: 0.3311 - val_accuracy: 0.8579

# Check agreement test

In [ ]:
import matplotlib.pylab as pylab

params = {'legend.fontsize': 'x-large',
         'axes.labelsize': '150',
         'axes.titlesize':'40',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}
pylab.rcParams.update(params)

plt.figure(figsize=(16,10))

plt.xlabel('Epochs',size=40)
plt.ylabel("Accuracy",size=40)
val = plt.plot(history.epoch, history.history['val_'+'accuracy'],
             label='Default variables', linewidth = 4)
plt.plot(historym.epoch, historym.history['val_'+'accuracy'],
             label='Feature engineering', linewidth = 4)

plt.legend(prop={'size':30})
plt.xticks(fontsize = 30)
plt.xticks(np.arange(0, max(history.epoch)+10, 50.0))
plt.yticks(fontsize = 30)
#plt.yticks(np.arange(romin(history.history['val_'+'accuracy']), max(history.history['val_'+'accuracy']), 2))
plt.xlim([0,max(history.epoch)+1])
plt.show()
plt.savefig('test.png', bbox_inches='tight')

In [16]:
check_agreement = pandas.read_csv(folder + 'check_agreement.csv.zip', index_col='id')
check_agreement = add_features(check_agreement)
check_agreement1 = check_agreement[variables_mod].to_numpy()
agreement_probs = model1m.predict(check_agreement1)
print(agreement_probs)
ks = evaluation.compute_ks(
    agreement_probs[check_agreement['signal'].values == 0],
    agreement_probs[check_agreement['signal'].values == 1],
    check_agreement[check_agreement['signal'] == 0]['weight'].values,
    check_agreement[check_agreement['signal'] == 1]['weight'].values)
print('KS metric', ks, ks < 0.09)

[[0.20729282]
 [0.04474391]
 [0.2827751 ]
 ...
 [0.9419084 ]
 [0.98079497]
 [0.9745996 ]]
KS metric 0.09081044553701173 False


# Check correlation test

In [17]:
check_correlation = pandas.read_csv(folder + 'check_correlation.csv.zip', index_col='id')
check_correlation = add_features(check_correlation)
check_correlation1 = check_correlation[variables_mod].to_numpy()
correlation_probs = model1m.predict(check_correlation1).squeeze()

cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
print('CvM metric', cvm, cvm < 0.002)

CvM metric 0.0010424020490809622 True


# Compute weighted AUC on the training data with min_ANNmuon > 0.4

In [18]:
train_eval = data[data['min_ANNmuon'] > 0.4]
train_probs = model1m.predict(train_eval[variables_mod].to_numpy()).squeeze()
AUC = sklearn.metrics.roc_auc_score(train_eval['signal'], train_probs)
print('AUC', AUC)

AUC 0.9242513534509336
